# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization with a dropout rate of p_drop = 0.5. I also train a model with Jacobian Regularization without dropout. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs (150 000 SGD iterations).

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
           Dropout-6                  [-1, 120]               0
            Linear-7                   [-1, 84]          10,164
           Dropout-8                   [-1, 84]               0
            Linear-9                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
---------------------------------------------

In [3]:
# Load models
dataset = "mnist"

model_names_set = [
    "model_no_reg",
    "model_no_reg_no_dropout",
    "model_l2",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,98.698 +/- 0.11
1,model_no_reg_no_dropout,98.842 +/- 0.086
2,model_l2,99.166 +/- 0.06
3,model_jacobi,98.66 +/- 0.112
4,model_jacobi_no_dropout,98.892 +/- 0.059
5,model_svb,98.442 +/- 0.058


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 50 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 30/30 [00:00<00:00, 116.19it/s]


In [7]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                       0.010  \
                                   mean conf_interval          mean   
model_no_reg_0               9397.10752   1640.019424  18925.979529   
model_no_reg_no_dropout_0   3124.085707    658.974539   8296.500406   
model_l2_0                  1872.659572    473.321832   1603.012042   
model_jacobi_0              9696.989582   1743.480779  16636.118235   
model_jacobi_no_dropout_0   3239.269016    589.626918   7118.733407   
model_svb_0                 1011.282659    268.295487   2021.521467   
model_no_reg_1             10802.936089   1812.477078  13164.218153   
model_no_reg_no_dropout_1   3543.187894    673.046513   5829.650145   
model_l2_1                  1884.946385    381.038664    1555.54639   
model_jacobi_1              8449.297505   1466.668468    9450.64785   
model_jacobi_no_dropout_1   2655.748935     689.59338   7086.289034   
model_svb_1                 1173.045075     384.66414   2194.082245   
model_no_reg_2              4973.873534   1289.366941  17785.193789   
model_no_reg_no_dropout_2   3994.497639    716.946412   7539.962552   
model_l2_2                  1847.384727    396.927242   1895.090495   
model_jacobi_2              8255.418067   1624.256957  10794.901942   
model_jacobi_no_dropout_2   3010.911888    676.674848   6415.291596   
model_svb_2                 1895.660934    488.528652   3183.103351   
model_no_reg_3             10517.952085   2138.351443  20499.784134   
model_no_reg_no_dropout_3   2748.737049    588.807065   6732.502937   
model_l2_3                  2535.188123    556.919598    4056.51978   
model_jacobi_3             12545.185898   1160.311551  11316.730161   
model_jacobi_no_dropout_3     2795.2954    614.348526   6568.532079   
model_svb_3                 1405.849444    330.270368   2483.356352   
model_no_reg_4             10730.165223   2460.654084  22952.031118   
model_no_reg_no_dropout_4   2049.195631    618.769219   7708.223593   
model_l2_4                  1907.007496    478.665395     3340.3785   
model_jacobi_4             10107.744282   2337.973166  22941.549955   
model_jacobi_no_dropout_4   2988.029358    581.146665   5727.451545   
model_svb_4                 1162.924134    367.639775   2386.824514   

                                                0.001                
                          conf_interval          mean conf_interval  
model_no_reg_0              1783.770986   22686.04441    2174.51603  
model_no_reg_no_dropout_0    993.379102  21668.074487     2298.8843  
model_l2_0                   413.092615    432.855148     164.06803  
model_jacobi_0              2089.201682  20336.666596   1643.048693  
model_jacobi_no_dropout_0    896.196344  15433.842996   2180.996834  
model_svb_0                  306.507966   4872.970497    468.738745  
model_no_reg_1              1995.561014   8091.816127   1183.622132  
model_no_reg_no_dropout_1    1024.30521  18316.354124   2236.788994  
model_l2_1                   304.526018    341.913251     180.73668  
model_jacobi_1              1385.333222    4652.56101    949.532128  
model_jacobi_no_dropout_1    751.616747  21671.276215   1724.736831  
model_svb_1                   292.14519   2740.501144    367.131011  
model_no_reg_2              2237.908353  32943.054795   1982.232028  
model_no_reg_no_dropout_2    779.361271  18703.517639    1671.35513  
model_l2_2                   363.555966   1431.110653    375.425175  
model_jacobi_2              1967.510959   3210.570785    807.816926  
model_jacobi_no_dropout_2    686.587094  15392.582303   1408.043634  
model_svb_2                  406.068888   5743.867735    551.489488  
model_no_reg_3              2146.405799  21786.315133   2168.883177  
model_no_reg_no_dropout_3    714.400164   18421.89869    1958.04492  
model_l2_3                   453.057439    8476.35314     654.99931  
model_jacobi_3              1374.057255   7007.510835    1428.63618  
model_jacobi_no_dropout_3    752.721466  17058.705955   1406.142214  
model_svb_3

#### Anisotropic Total Variation

In [8]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 30/30 [00:00<00:00, 115.58it/s]


In [9]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                   0.010                \
                               mean conf_interval      mean conf_interval   
model_no_reg_0             10407.96   1815.995682  21156.56   1997.586641   
model_no_reg_no_dropout_0   3432.24    723.978823    9113.2   1094.997792   
model_l2_0                   2060.5    521.329695   1763.38    455.810658   
model_jacobi_0             10737.42   1924.716843  18589.42   2338.149357   
model_jacobi_no_dropout_0   3567.42    653.630693   7826.64    981.085676   
model_svb_0                 1111.36    290.403137   2216.64    339.738177   
model_no_reg_1             11967.72    2004.08372  14724.98   2228.478639   
model_no_reg_no_dropout_1   3905.94    743.117657   6426.66   1132.724448   
model_l2_1                   2050.3    414.876987   1700.02    331.135046   
model_jacobi_1              9345.58   1626.750529  10519.38   1534.993653   
model_jacobi_no_dropout_1   2925.04    761.790743   7824.08    831.641791   
model_svb_1                  1276.2    412.428297   2394.44    316.150817   
model_no_reg_2              5507.68   1419.439906   19884.4   2501.852593   
model_no_reg_no_dropout_2   4379.58    785.708753   8278.38    845.798565   
model_l2_2                   2017.1    436.568735   2076.26    397.424454   
model_jacobi_2              9116.22   1794.374355  11988.46   2186.526608   
model_jacobi_no_dropout_2    3292.9    740.147565   7040.64    743.896813   
model_svb_2                 2073.04     532.96163   3487.06    439.976761   
model_no_reg_3             11664.54   2374.558599  22966.62   2419.947996   
model_no_reg_no_dropout_3   3025.42    650.537562   7397.34    782.087198   
model_l2_3                  2787.94    611.018357   4471.16    497.359503   
model_jacobi_3              13883.9   1283.186364  12607.46   1528.762066   
model_jacobi_no_dropout_3   3069.66     679.87188   7238.56    835.912338   
model_svb_3                 1542.56    359.581235    2726.8    293.765721   
model_no_reg_4             11919.28   2731.777617  25684.36    2776.29577   
model_no_reg_no_dropout_4   2237.44    674.419336   8483.68   1063.876965   
model_l2_4                   2088.6    525.305961   3670.88    466.303395   
model_jacobi_4              11238.8    2604.33719  25710.04   2314.736563   
model_jacobi_no_dropout_4   3281.42      637.4119   6292.96     662.67009   
model_svb_4                 1288.72    402.771857   2626.14    359.217594   

                              0.001                
                               mean conf_interval  
model_no_reg_0              25879.1   2430.278673  
model_no_reg_no_dropout_0  24208.38   2570.465619  
model_l2_0                   466.82    172.025516  
model_jacobi_0             23125.68   1897.126745  
model_jacobi_no_dropout_0  17425.22   2477.425715  
model_svb_0                 5429.64    521.983604  
model_no_reg_1              9174.44   1345.624787  
model_no_reg_no_dropout_1  20451.24   2492.326827  
model_l2_1                   378.34    204.884075  
model_jacobi_1              5255.16   1068.355275  
model_jacobi_no_dropout_1   24259.1   1966.766784  
model_svb_1                 3021.16    404.525901  
model_no_reg_2             37770.84   2301.719262  
model_no_reg_no_dropout_2  20938.48   1851.709561  
model_l2_2                  1563.26    401.317867  
model_jacobi_2              3624.22    906.851118  
model_jacobi_no_dropout_2   17087.0    1570.73318  
model_svb_2                  6353.0    616.381261  
model_no_reg_3             24678.08   2433.220054  
model_no_reg_no_dropout_3  20482.52   2138.603944  
model_l2_3                  9428.48    729.835105  
model_jacobi_3              7972.84    1663.68402  
model_jacobi_no_dropout_3  19083.54   1595.260091  
model_svb_3                  3462.5     410.48002  
model_no_reg_4             45009.62   2341.354738  
model_no_reg_no_dropout_4  29180.28   2073.648926  
model_l2_4                  8914.74    890.073491  
model_jacobi_4             50077.14   2653.356366  
model